In [1]:
%load_ext autoreload


In [2]:
%autoreload 2

In [5]:
import pandas as pd
import numpy as np
from scipy import stats
from datetime import datetime
''' Sn properties - Masses, Ratios, Cup Configurations, ... '''
from sn_config import *
'''Classes for Reading in the Data and applying Internal Normalisation'''
from nu_data_reduction import NU_data_read, int_norm, evaluation
from pylab import plot, show, savefig
from scipy import stats
import os.path
import re
#%pylab inline

In [32]:
path = "/Volumes/friebelm/PhD/NU Plasma/Measurements/2015-03-25/"
path_sav = "/Volumes/friebelm/PhD/NU Plasma/Measurements/compilation/"
path_all = "/Volumes/friebelm/PhD/NU Plasma/Measurements/2016-07-16/"
files_start = 4547
files_end = 4713


In [33]:
# cup configuration
#cup_config = cycle_Sb
cup_config = cycles2
# Isotopes used for Interference correction
corr_isotopes_1 = {"Cd" : "111", "Te" : "126"}
corr_isotopes_2 = {"Cd" : "111","Te" : "125", "Xe" : "129"}
corr_isotopes_Sb = {"Te": "125"}
# Mass Range of cup configuration
mass_range = cycles2_mass_range
#isotopes = [["111", "112", "114", "115", "116", "117", "118", "119", "122", "124"], ["117", "118", "119", "122", "124"]]
isotopes = [["111", "112", "114", "115", "116", "117", "118", "119", "122", "124", "125", "126", "129"]]
#isotopes = [["116", "117", "118", "119", "122", "124"]]
denom_isotope = "120"
isotope_column = ["111", "112", "114", "115", "116", "117", "118", "119", "122", "124", "125", "126", "129"]
data_sample_column = [(i + "/" + "120") for i in isotope_column]
print data_sample_column
#number of iterations for beta
iter_beta = 10

# Interference_corr on the denominator isotope
isotope_denom_corr = True

blk_corr = True

['111/120', '112/120', '114/120', '115/120', '116/120', '117/120', '118/120', '119/120', '122/120', '124/120', '125/120', '126/120', '129/120']


In [34]:
filenames = os.listdir(path_all)

In [35]:
blk_ls = []
sample_ls = []
std_ls = []
for i in filenames:
    sample = re.search(r""+"Data_"+"(.*).csv", i)
    if sample == None:
        None
    else:
        sample = sample.group(1)
        df = NU_data_read(path_all, sample, cup_config)
        sample_name = df.extract_metadata(sample, "Sample Name")
        if sample_name == "blank sol" or sample_name == "wash" or sample_name == "wash clean" or sample_name == "Teflon blk":
            blk_ls.append(sample)
        elif sample_name == "SPEX 100ppb" or sample_name == "NIST 100ppb":
            std_ls.append(sample)
        else:
            sample_ls.append(sample)

In [41]:
columns = ["cycle", "sample", "date","H10 (1)", "H9 (1)", "H8 (1)", "H7 (1)", "H6 (1)", "H5 (1)", "H4 (1)", "H3 (1)", "H2 (1)", "H1 (1)", "Ax (1)", "L1 (1)", "L2 (1)", "L3 (1)", "L4 (1)", 
             "H10 (2)", "H9 (2)", "H8 (2)", "H7 (2)", "H6 (2)", "H5 (2)", "H4 (2)", "H3 (2)", "H2 (2)", "H1 (2)", "Ax (2)", "L1 (2)", "L2 (2)", "L3 (2)", "L4 (2)"]
df_all = pd.DataFrame(columns = columns)
for sample in blk_ls:
    df = NU_data_read(path_all, sample, cup_config)
    datafile = "Data_" + str(sample) + ".csv"
    data = pd.read_csv(path_all + datafile, skiprows=58, index_col='Cycle')
    if "H8 (2)" in data.columns:
        data_zero = data.ix[:,"H10 (Z1)":"L4 (Z1)"]
        data_1 = data.ix[:,"H10 (1)":"L4 (1)"]
        data_2 = data.ix[:,"H10 (2)":"L4 (2)"]
        data_zero.columns = data_1.columns
        data_1_corr = data_1 - data_zero
        data_1_corr["cycle"] = data_1_corr.index.values
        data_zero.columns = data_2.columns
        data_2_corr = data_2 - data_zero
        data_2_corr["cycle"] = data_2_corr.index.values
        data_1_corr = data_1_corr.merge(data_2_corr)
        data_1_corr = mad_outlier_rejection(data_1_corr)
    else:
        data_zero = data.ix[:,"H10 (Z1)":"L4 (Z1)"]
        data_1 = data.ix[:,"H10 (1)":"L4 (1)"]
        data_zero.columns = data_1.columns
        data_1_corr = data_1 - data_zero
        data_1_corr["cycle"] = data_1_corr.index.values
        data_1_corr = mad_outlier_rejection(data_1_corr)
        
    sample_name = df.extract_metadata(sample, "Sample Name")
    date = df.extract_metadata(sample, "Date")
    starttime = df.extract_metadata(sample, "Start Time")
    data_1_corr["date"] = datetime.strptime(date+starttime, '%d/%m/%Y%H:%M')
    data_1_corr["sample"] = sample_name
    df_all = pd.concat([df_all,data_1_corr], axis=0, ignore_index=True)
    #df_all = df_all.append(data_1_corr)
    df_all = df_all[columns]
df_all.set_index("date", inplace = True)
df_all

,cycle,sample,H10 (1),H9 (1),H8 (1),H7 (1),H6 (1),H5 (1),H4 (1),H3 (1),...,H5 (2),H4 (2),H3 (2),H2 (2),H1 (2),Ax (2),L1 (2),L2 (2),L3 (2),L4 (2)
date,,,,,,,,,,,,,,,,,,,,,
2016-07-16 13:20:00,1,blank sol,0,0,0.000353,0.000264,0.002275,0.000510,0.001384,0.000478,...,0.000056,2.452250e-05,0.000369,-0.000006,0.000260,0.000022,0.002237,0.000474,0.001353,0.000402
2016-07-16 13:20:00,2,blank sol,0,0,0.000394,0.000316,0.002506,0.000554,0.001529,0.000535,...,0.000063,-5.475800e-06,0.000402,0.000017,0.000272,0.000034,0.002458,0.000515,0.001483,0.000428
2016-07-16 13:20:00,3,blank sol,0,0,0.000433,0.000261,0.002416,0.000565,0.001486,0.000460,...,0.000082,1.238020e-05,NaN,0.000019,NaN,0.000043,NaN,NaN,NaN,NaN
2016-07-16 13:20:00,4,blank sol,0,0,NaN,0.000394,NaN,NaN,NaN,0.000628,...,0.000059,1.666570e-05,0.000356,0.000022,0.000278,0.000031,0.002396,0.000527,0.001425,0.000470
2016-07-16 13:20:00,5,blank sol,0,0,0.000397,0.000289,0.002348,0.000501,0.001394,0.000441,...,0.000074,7.380600e-06,0.000389,0.000019,0.000300,0.000032,0.002406,0.000511,0.001437,0.000453
2016-07-16 13:20:00,6,blank sol,0,0,0.000333,0.000260,0.002317,0.000513,0.001382,0.000462,...,0.000074,-1.833220e-05,0.000376,0.000024,0.000278,0.000006,0.002371,0.000480,0.001426,0.000454
2016-07-16 13:20:00,7,blank sol,0,0,0.000370,0.000266,0.002358,0.000531,0.001431,0.000470,...,0.000036,5.237800e-06,0.000341,-0.000031,0.000249,0.000039,0.002292,0.000463,0.001338,0.000443
2016-07-16 13:20:00,8,blank sol,0,0,0.000548,NaN,NaN,NaN,NaN,NaN,...,0.000036,-4.761000e-07,NaN,0.000022,NaN,0.000059,NaN,NaN,NaN,NaN
2016-07-16 13:20:00,9,blank sol,0,0,0.000387,0.000292,0.002424,0.000505,0.001473,0.000480,...,0.000089,-1.333250e-05,0.000406,0.000063,0.000291,0.000072,0.002496,0.000555,0.001526,0.000461


In [42]:
def mean(x):
    mean = np.nanmean(x) 
    return mean

df_all_mean = df_all.drop("cycle", axis=1).groupby(df_all.index).agg(mean)
df_all_mean["sample"] = df_all[df_all["cycle"] == 1]["sample"]
df_all_mean

,H10 (1),H9 (1),H8 (1),H7 (1),H6 (1),H5 (1),H4 (1),H3 (1),H2 (1),H1 (1),...,H4 (2),H3 (2),H2 (2),H1 (2),Ax (2),L1 (2),L2 (2),L3 (2),L4 (2),sample
date,,,,,,,,,,,,,,,,,,,,,
2016-07-16 13:20:00,0,0,0.000403,0.000302,0.002432,0.000538,0.001465,0.000502,0.000986,0.000035,...,0.000004,0.000383,0.000019,0.000275,0.000041,0.002406,0.000514,0.001450,0.000454,blank sol
2016-07-16 14:39:00,0,0,0.000931,0.000713,0.005479,0.001307,0.003645,0.001151,0.002277,0.000057,...,0.000015,0.000918,0.000037,0.000700,0.000030,0.005346,0.001262,0.003567,0.001131,blank sol
2016-07-16 15:57:00,0,0,0.001115,0.000845,0.006334,0.001547,0.004289,0.001347,0.002628,0.000070,...,0.000011,0.001123,0.000018,0.000869,0.000031,0.006453,0.001578,0.004373,0.001367,blank sol
2016-07-16 17:16:00,0,0,0.001349,0.001030,0.007645,0.001887,0.005248,0.001652,0.003312,0.000086,...,0.000019,0.001424,0.000031,0.001092,0.000035,0.008039,0.001974,0.005505,0.001727,blank sol
2016-07-16 18:46:00,0,0,0.001099,0.000826,0.006305,0.001531,0.004273,0.001341,0.002622,0.000065,...,0.000019,0.001032,0.000031,0.000794,0.000039,0.006022,0.001456,0.004064,0.001315,blank sol
2016-07-16 20:07:00,0,0,0.001077,0.000823,0.006235,0.001504,0.004145,0.001317,0.002556,0.000072,...,0.000007,0.001112,0.000029,0.000857,0.000048,0.006316,0.001523,0.004189,0.001343,blank sol
2016-07-16 21:27:00,0,0,0.002114,0.001621,0.011678,0.002914,0.008061,0.002528,0.004814,0.000132,...,0.000022,0.002326,0.000030,0.001802,0.000050,0.012857,0.003211,0.008896,0.002770,blank sol
2016-07-16 22:42:00,0,0,0.003259,0.002504,0.017800,0.004515,0.012495,0.003917,0.007417,0.000173,...,0.000006,0.003103,0.000030,0.002401,0.000036,0.017070,0.004313,0.011967,0.003743,blank sol
2016-07-16 22:49:00,0,0,0.002157,0.001646,0.011939,0.002982,0.008272,0.002601,0.004967,0.000123,...,0.000009,0.002178,0.000036,0.001713,0.000038,0.012299,0.003091,0.008537,0.002677,Teflon blk


In [40]:
df_peak = pd.DataFrame(columns = ["date", "sample", "peak_center_mass"])
data_1_corr = pd.DataFrame()
for sample in blk_ls:
    df = NU_data_read(path_all, sample, cup_config)
    datafile = "Results_" + str(sample) + ".txt"
    sample_name = df.extract_metadata(sample, "Sample Name:")
    date = df.extract_metadata(sample, "Date and Time:")
    peak_center = df.extract_metadata(sample, "Axial Mass(es) used:")
    data_1_corr["date"] = datetime.strptime(date, '%d/%m/%Y%H:%M')
    data_1_corr["sample"] = sample_name
    data_1_corr["peak_center_mass"] = peak_center
    df_peak = pd.concat([df_peak,data_1_corr], axis=0, ignore_index=True)

df_peak

UnboundLocalError: local variable 'metadata' referenced before assignment

In [23]:
df_all.to_csv(path_all + "blanks_no_outlier.csv", mode='w', header=True)

In [25]:
df_all_mean.to_csv(path_all + "blanks_mean_no_outlier.csv", mode='w', header=True)

In [229]:
df_internal_norm_mean_sd.to_csv(path + "Sn_H8_L4_2_cycles_internal_norm_mean_no_bgd_corr.csv")

In [389]:
epsilon.to_csv(path + "Sn_H8_L4_2_cycles_internal_norm_epsilon_bgd_corr.csv")